# Kaggle TPS 2021 October Autogluon

## Check Colab environment
### GPU

In [ ]:
# Google Colab
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Oct  2 07:23:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### High RAM

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


## PIP Installs
### Uninstall lightgbm to use GPU version

**There will be a prompt to confirm**
* Say y

In [ ]:
!pip uninstall lightgbm


Found existing installation: lightgbm 2.2.3
Uninstalling lightgbm-2.2.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/lightgbm-2.2.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/lightgbm/*
Proceed (y/n)? y
  Successfully uninstalled lightgbm-2.2.3


Download the code and do the build for LightGBM GPU version

In [ ]:
! git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;    


Cloning into 'LightGBM'...
remote: Enumerating objects: 23463, done.
remote: Counting objects: 100% (1142/1142), done.
remote: Compressing objects: 100% (615/615), done.
remote: Total 23463 (delta 710), reused 816 (delta 503), pack-reused 22321
Receiving objects: 100% (23463/23463), 18.07 MiB | 25.04 MiB/s, done.
Resolving deltas: 100% (17109/17109), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21733, done.        
remote: Counting objects: 100% (5/5), done.        
remote: 

### Autogluon-Colab GPU compatibility

In [ ]:
!pip uninstall -y mkl
!pip install --upgrade mxnet-cu100
!pip install autogluon
!pip install -U ipykernel

Found existing installation: mkl 2019.0
Uninstalling mkl-2019.0:
  Successfully uninstalled mkl-2019.0
     |████████████████████████████████| 352.6 MB 5.4 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 273 kB 15.5 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 352 kB 88.8 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 4.2 MB 85.3 MB/s 
     |████████████████████████████████| 131 kB 73.8 MB/s 
     |████████████████████████████████| 206 kB 75.2 MB/s 
     |████████████████████████████████| 786 kB 83.9 MB/s 
     |████████████████████████████████| 22.3 MB 83.3 MB/s 
     |████████████████████████████████| 27.4 MB 1.2 MB/s 
     |████████████████████████████████| 1.3 MB 57.3 MB/s 
     |████████████████████████████████| 

     |████████████████████████████████| 124 kB 15.9 MB/s 
     |████████████████████████████████| 788 kB 91.1 MB/s 
     |████████████████████████████████| 370 kB 88.2 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 req

**After the PIP install it will tell you to restart runtime.  You DON'T need to**

## Imports

In [ ]:
import random
import os
from google.colab import output

import catboost as cb
from catboost import CatBoostRegressor, Pool, CatBoostClassifier

import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, \
                            AdaBoostClassifier,AdaBoostRegressor, \
                            VotingClassifier, \
                            RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor # can use KNeighborsClassifier as regressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
#from sklearn import datasets
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import pickle
from autogluon.tabular import TabularPredictor

## Google Drive Mount

This is to make it easy to access the data files

Note that autogluon will require a significant amount of space as well while it generates and fits models.  **If you don't have the space on Google Drive, it may be better to upload your data files and avoid mounting the drive.**

In [ ]:
working_dir = 'MyDrive/kaggle/202110'
drive_dir = '/content/drive/'
model_dir = 'models'
output_dir = 'output'
data_dir = 'data'
from google.colab import drive
drive.mount(drive_dir)  # Do this once only

mounted_drive = f'{drive_dir}/{working_dir}' #''
os.chdir(mounted_drive)

Mounted at /content/drive/


The data files were massaged in the Data Prep Notebook

In [ ]:
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)



Note that X still has the target column, since Autogluon expects to ingest a dataframe that contains both the target and the training data.

In [ ]:
X = pd.read_csv(f'./{data_dir}/reduced_mem_imputed.csv')
test_X = pd.read_csv(f'./{data_dir}/reduced_mem_imputed_test.csv') # this is the Kaggle test dataset

In [ ]:
TARGET = 'target'
N_SPLITS=5
SEED=42

Here we fit the model without the id, otherwise Tree models make take that into account.

In [ ]:

X = X.drop(columns = ['id'])


## Autogluon

### Default Parameters

Below are the default Autogluon Parameters.  

You can prevent a set of models from being tuned by excluding the parameters or by limiting the time spent running the fitting.  

For this run, KNN is explicitly going to be excluded since it takes several hours.

---
```
hyperparameters = {
               'NN': {'num_epochs': 500},
               'GBM': {'num_boost_round': 10000},
               'CAT': {'iterations': 10000},
               'RF': {'n_estimators': 300},
               'XT': {'n_estimators': 300},
               'KNN': {},
               'custom': ['GBM'],
}
```
---


In [ ]:
train_data = X
eval_metric = 'roc_auc' 
num_seconds = 15 * 3600  # initially this was set to 10 but K neighbors was still running after the 6 it initially estimated - set it to 7, but it didn't do all the models, although some i didn't really want

### Run Predictor

Note that AutoGluon saves the files to disk, so if the notebook is still active, the predictor can be reloaded and run.

In [ ]:
predictor = TabularPredictor(label=TARGET, eval_metric=eval_metric, path='./autogluon/', verbosity=2).fit(
    train_data, presets='best_quality', time_limit=num_seconds, hyperparameters = {'GBM': {'num_boost_round': 20000}, 'CAT': {'iterations': 20000}, 'XGB':{'n_estimators':20000}}
)
# Kaggle 0.85612 hyperparameters = {'GBM': {'num_boost_round': 20000}, 'CAT': {'iterations': 20000}, 'XGB':{'n_estimators':20000}

Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 54000s
AutoGluon will save models to "./autogluon/"
AutoGluon Version:  0.3.1
Train Data Rows:    1000000
Train Data Columns: 287
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
NumExpr defaulting to 8 threads.
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    50899.18 MB
	Train Data (Original)  Memory Usage: 2296.0 MB (4.5% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the feat

[1000]	train_set's binary_logloss: 0.442692	valid_set's binary_logloss: 0.467297


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's binary_logloss: 0.442594	valid_set's binary_logloss: 0.468089


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's binary_logloss: 0.442638	valid_set's binary_logloss: 0.468109


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's binary_logloss: 0.443016	valid_set's binary_logloss: 0.464488


	0.8567	 = Validation score   (roc_auc)
	12134.95s	 = Training   runtime
	69.83s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 12453.45s of the 30447.08s of remaining time.
	0.8562	 = Validation score   (roc_auc)
	8323.52s	 = Training   runtime
	7.37s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ... Training model for up to 10263.52s of the 28257.14s of remaining time.
	0.856	 = Validation score   (roc_auc)
	7616.21s	 = Training   runtime
	56.47s	 = Validation runtime
Completed 3/20 k-fold bagging repeats ...
Fitting model: WeightedEnsemble_L2 ... Training model for up to 3596.03s of the 25707.99s of remaining time.
	0.8568	 = Validation score   (roc_auc)
	127.97s	 = Training   runtime
	0.43s	 = Validation runtime
Fitting 3 L2 models ...
Fitting model: LightGBM_BAG_L2 ... Training model for up to 25579.51s of the 25578.22s of remaining time.
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is 

## Kaggle Submission File

In [ ]:
test_X = test_X.drop(columns = ['id'])

y_predproba = predictor.predict_proba(test_X)

y_predproba = pd.DataFrame(y_predproba)
y_predproba['id'] = y_predproba.index + len(X)
y_predproba.to_csv('./artifacts/autogluon_predproba.csv', index=False)
results_df = y_predproba.rename(columns={1:'target'}).copy()
results_df = results_df[['id','target']]
results_df.to_csv('./artifacts/autogluon_submission.csv', index=False)